<a href="https://colab.research.google.com/github/sam505/Machine_Learning/blob/master/PYTORCH_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torchvision
import torch
import torch.nn as nn
from torch import optim
import torch.utils.data
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([transforms.Resize((28,28)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,))
                               ])

train_batch_size = 64
test_batch_size = 128

mnist_train = torchvision.datasets.MNIST(root="~/", 
                                         download=True, 
                                         transform=transform, 
                                         train=True)

mnist_test = torchvision.datasets.MNIST(root="~/", 
                                        download=True, 
                                        transform=transform, 
                                        train=False)

train_loader = torch.utils.data.DataLoader(mnist_train,
                                          shuffle=True,
                                          batch_size=train_batch_size,
                                          num_workers=2)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          shuffle=False,
                                          batch_size=test_batch_size,
                                          num_workers=2)


Extracting /root/MNIST/raw/train-images-idx3-ubyte.gz to /root/MNIST/raw




Extracting /root/MNIST/raw/train-labels-idx1-ubyte.gz to /root/MNIST/raw




Extracting /root/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/MNIST/raw




Extracting /root/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
len(mnist_train)

60000

In [4]:
len(mnist_test)

10000

In [5]:
learning_rates = [0.1, 0.01, 0.001]

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
def train(model, optimizer, epoch, train_loader, log_interval):
    # State that you are training the model
    model.train()
    correct = 0
    count = 1
    # define loss function
    loss_fn = nn.CrossEntropyLoss()

    # Iterate over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Wrap the input and target output in the `Variable` wrapper
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
            data = data.to("cuda")
            target = target.to("cuda")

        # Clear the gradients, since PyTorch accumulates them
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        # Backward propagation
        loss.backward()

        # Update the parameters(weight,bias)
        optimizer.step()
        
        predicted = torch.max(output.data, 1)[1] 
        correct += (predicted == target).sum()

        # print log
        if batch_idx % log_interval == 0:
            train_accuracy = correct / ((batch_idx+1)*len(data))
            train_loss = loss.data.item()
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                train_loss, train_accuracy))
        
    return loss.data.item(), train_accuracy

In [8]:
def test(model, epoch, test_loader):
    # State that you are testing the model; this prevents layers e.g. Dropout to take effect
    model.eval()

    # Init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    # define loss function
    loss_fn = nn.CrossEntropyLoss(size_average=False)

    # Iterate over data
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
            data = data.to("cuda")
            target = target.to("cuda")

        # Forward propagation
        output = model(data)

        # Calculate & accumulate loss
        test_loss += loss_fn(output, target).data.item()

        # Get the index of the max log-probability (the predicted output label)
        pred = np.argmax(output.cpu().data, axis=1)

        # If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.cpu().data).sum()

    # Print log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correct / len(test_loader.dataset)

In [9]:
# Provide seed for the pseudorandom number generator s.t. the same results can be reproduced
for learning_date in learning_rates:
    torch.manual_seed(123)

    model = LeNet()

    check_gpu = torch.cuda.is_available()
    if check_gpu:
        model = model.cuda()
        print ('USING GPU')
    else:
        print ('USING CPU')
    epochs = 10
    log_interval = 100
    writer = SummaryWriter("runs/Mnist_Adam_{}".format(learning_date))
    optimizer = optim.Adam(model.parameters(), lr=learning_date)
    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train(model, optimizer, epoch, train_loader, log_interval=log_interval)
        test_loss, test_acc = test(model, epoch, test_loader) 
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/test', test_acc, epoch)

USING GPU
Train set, Epoch 1 [0/60000 (0%)]	Loss: 2.302851	Accuracy: 0.20
Train set, Epoch 1 [6400/60000 (11%)]	Loss: 2.277897	Accuracy: 0.11
Train set, Epoch 1 [12800/60000 (21%)]	Loss: 2.322655	Accuracy: 0.11
Train set, Epoch 1 [19200/60000 (32%)]	Loss: 2.299893	Accuracy: 0.11
Train set, Epoch 1 [25600/60000 (43%)]	Loss: 2.313671	Accuracy: 0.11
Train set, Epoch 1 [32000/60000 (53%)]	Loss: 2.326670	Accuracy: 0.11
Train set, Epoch 1 [38400/60000 (64%)]	Loss: 2.335364	Accuracy: 0.11
Train set, Epoch 1 [44800/60000 (75%)]	Loss: 2.319967	Accuracy: 0.11
Train set, Epoch 1 [51200/60000 (85%)]	Loss: 2.311900	Accuracy: 0.11
Train set, Epoch 1 [57600/60000 (96%)]	Loss: 2.364736	Accuracy: 0.11


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 2.3066, Accuracy: 958/10000 (9.58%)

Train set, Epoch 2 [0/60000 (0%)]	Loss: 2.309947	Accuracy: 0.05
Train set, Epoch 2 [6400/60000 (11%)]	Loss: 2.320487	Accuracy: 0.11
Train set, Epoch 2 [12800/60000 (21%)]	Loss: 2.300097	Accuracy: 0.11
Train set, Epoch 2 [19200/60000 (32%)]	Loss: 2.307399	Accuracy: 0.11
Train set, Epoch 2 [25600/60000 (43%)]	Loss: 2.346181	Accuracy: 0.11
Train set, Epoch 2 [32000/60000 (53%)]	Loss: 2.290435	Accuracy: 0.11
Train set, Epoch 2 [38400/60000 (64%)]	Loss: 2.255812	Accuracy: 0.11
Train set, Epoch 2 [44800/60000 (75%)]	Loss: 2.315505	Accuracy: 0.11
Train set, Epoch 2 [51200/60000 (85%)]	Loss: 2.339714	Accuracy: 0.11
Train set, Epoch 2 [57600/60000 (96%)]	Loss: 2.306776	Accuracy: 0.11

Test set, Epoch 2 , Average loss: 2.3081, Accuracy: 1135/10000 (11.35%)

Train set, Epoch 3 [0/60000 (0%)]	Loss: 2.309682	Accuracy: 0.14
Train set, Epoch 3 [6400/60000 (11%)]	Loss: 2.337271	Accuracy: 0.11
Train set, Epoch 3 [12800/60000 (21%)]

In [10]:
len(mnist_train)

60000